In [173]:
import pandas as pd 
from escape import read_wildSequence
import copy
import Escape_score_predictor as esp
import matplotlib.pyplot as plt
import numpy as np
escape_threshold = 0.95

sig_escape_file ="/home/perm/cov/data/additional_escape_variants/gen/SINGLE_RES_SUB.csv"
wild_seq = "/home/perm/cov/data/cov2_spike_wt.fasta"
escape_server_csv_path = "/home/perm/cov/data/esc_server_seq.csv"
escape_csv_path_with_scores = "/home/perm/cov/data/esc_server_seq_with_esc_scores.csv"
escape_position_file = "/home/perm/cov/data/additional_escape_variants/gen/signficant_positions.csv"
insicio_mutated_file_path = "/home/perm/cov/data/additional_escape_variants/gen/insicio_mutated_spikes.csv"
insicio_mutated_file_escape_path = "/home/perm/cov/data/additional_escape_variants/gen/insicio_mutated_spikes_with_scores.csv"
escape_spike_save_path= "/home/perm/cov/data/additional_escape_variants/gen/insicio_mutated_escape_spikes.csv"
known_mutants_case1_path = "/home/perm/cov/data/mut_cases/case1.csv"
known_mutants_case2_path = "/home/perm/cov/data/mut_cases/case2.csv"
known_mutants_case1_path_with_score = "/home/perm/cov/data/mut_cases/case1_scores.csv"
known_mutants_case2_path_with_scores = "/home/perm/cov/data/mut_cases/case2_scores.csv"


hex_plot_save_path = "/home/perm/cov/data/analysis/escape_gan/insilico_mutant_escape_analysis.png"
residues = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 
    'F', 'P', 'S', 'T', 'W', 'Y', 'V']
def generate_escape_server_escape_segments():
    seq = str(read_wildSequence())
    wild_seq_array = list(seq)
    wild_seq_len = len(wild_seq_array)
    df = pd.read_csv(sig_escape_file)
    mutants = df.iloc[:, 0].to_list()
    df1 = pd.DataFrame()
    for mutant in mutants:
        position = int(mutant[1:-1] )
        original_residue = mutant[0]
        changed_to = mutant[-1]
        # print(f"{mutant} original: {original_residue} Position: {position} Changed to : {changed_to}")
        segment_len = 20 
        position_index  = position - 1
        wild_seq_array_copy = copy.deepcopy(wild_seq_array)
        wild_seq_array_copy[position_index] = changed_to

        #Extract 20 length segment
        extract_start_position = position_index - int(segment_len/2)
        extract_end_position = position_index + int(segment_len/2) #end postion should be exclusive
        if extract_start_position < 0 : 
            extract_start_position = 0
            extract_end_position = segment_len
        if extract_end_position > wild_seq_len:
            extract_end_position = wild_seq_len + 1 # end position is exclusive so, if 1 not added end residue will not come
            extract_start_position = wild_seq_len - segment_len +1

        seq_segement = wild_seq_array_copy[extract_start_position: extract_end_position]
        assert len(seq_segement) == 20
        data = {"window_seqs":"".join(seq_segement), "position" : str(position), "mutant" : mutant }
        df1 = df1.append(data, ignore_index=True) #Append row | prevents duplicate indices and assigns new consecutive indices
        
        #Clear the exiting wild_seq_array_copy
        wild_seq_array_copy = []

    #Save csv file 
    df1.to_csv(escape_server_csv_path, index=False)
    print("Escape Server escape sequence segments generated !!")
    
def generate_escape_scores(input_csv_esc_path, output_csv_esc_path):
    '''
        This method predicts the scores given input spikes in csv format 
        frome escape csv file and generated new csv file with escape scores appended
    '''
    raw_scores = esp.predict_escape_score(input_csv_esc_path)
    outputs = raw_scores.flatten() #reduce dimension to 1d eg outputs (2040, 1) to 2040, 
    #read the orginal input_csv_esc_path file and create new file with appended score
    df2 = pd.read_csv(input_csv_esc_path)
    #Add escape scores
    df2['escape_scores'] = outputs
    df2.to_csv(output_csv_esc_path, index=False)
    print("Escape scores saved successfully to csv path:", output_csv_esc_path)
    
def extract_sequence(input_seq, segment_len, position):
    '''
    Given any sequences of Lenght L >= Segment length, extract the segment from the sequence keeping position in the middle 
    @param
    input_seq : sequence in string 
    returns Segment and its start postion
    '''
    assert position < len(input_seq), "Position must be less than input sequence length"
    position = position - 1 #actual index position
    seq_array = list(input_seq) #Convert string sequence to array 
    start_pos = position-int(segment_len/2)
    end_position = position+ int(segment_len/2)
    if start_pos < 0 :
        start_pos = 0
        end_position = segment_len
    if end_position > len(input_seq):
        end_position = len(input_seq) #Range will exclusive so
        start_pos = len(input_seq) - segment_len
    
    segment_array = seq_array[start_pos : end_position]
    segment = "".join(segment_array)
  
    assert len(segment) == segment_len, f"Error occured while extracting sequence from pos: {position}"
    segment_start_position = start_pos + 1 #start position is an index as it starts from 0
    return segment, segment_start_position

def get_mutated_seqences(seq_seg, mut_position):
    '''
    mut_position : It initially holds start postion of the sequence segment
    '''
    #residues 
    mutated_seq_dict = {}
    mutated_index = 0 #We start mutating from oth index
    for original_residue in seq_seg:
        #Mutate this residue to all other residue and construct sequences
        for mutated_residue in residues:
            if  original_residue == mutated_residue:
                continue #Do not construct this sequence , as this will be a wild sequence
            #From 20 possible amino acid 19 muated sequences will be constructed for each position when excluding the orignal seg
            mutated_seq_array = list(seq_seg)
            mutated_seq_array[mutated_index] = mutated_residue
            
            mutated_seq = "".join(mutated_seq_array)
            mutated_seq_dict[mutated_seq] = original_residue+str(mut_position) + mutated_residue
            
       
        #Once all the 19 mutated sequences constructed for a particular postion next, the position of muation will be increased
        mutated_index = mutated_index + 1
        mut_position = mut_position + 1
     
    return mutated_seq_dict
        

def generate_insilico_muated_spikes(insicio_mutated_file_path):
    df = pd.read_csv(escape_position_file)
    df.head()
    escape_positions = df['pos'].to_list()

    wild_sequence = str(read_wildSequence()) 
    segment_len = 20
    i = 0 
    mutated_seq_dict = {}
    for position in escape_positions:
        segment, seg_start_pos  = extract_sequence(wild_sequence, segment_len, position)
        # print(segment, "Len:", len(segment), "Segment start postion:", seg_start_pos)
        seq_seg_dictionary = get_mutated_seqences(segment, seg_start_pos)
        #Add segmented dictionary for individual dictionary 
        mutated_seq_dict.update(seq_seg_dictionary)

    #Save the results to csv 
    df1 = pd.DataFrame.from_dict(mutated_seq_dict, orient='index', columns=['mutation'])
    #df1 = df1.transpose() #Transpose
    df1.index.name= "window_seqs"
    df1.to_csv(insicio_mutated_file_path)
    print("File generated successfully to path: ",insicio_mutated_file_path)  

def generate_hexbin_plot(hex_plot_save_path, escape_spike_save_path, escape_threshold=0.95):
    #Read scores 
    df1 = pd.read_csv(insicio_mutated_file_escape_path)
    filtered_dataframe = pd.DataFrame(columns=['window_seqs', 'mutation', 'escape_scores'])

    filtered_postions, filtered_scores = [], []
    non_sig_pos, non_sig_scores = [], []
    for index, record in df1.iterrows():
        mutation =  record['mutation']
        escape_score = record['escape_scores']
        spike = record['window_seqs']
        pos = int(mutation[1:-1]) #Start from first index, ie second postion and excluding the last index | G502A 
        
        if escape_score > escape_threshold :
            filtered_postions.append(pos)
            filtered_scores.append(escape_score)
            data = {'window_seqs': spike , 'mutation':mutation, 'escape_scores': escape_score}
            filtered_dataframe = filtered_dataframe.append(data, ignore_index= True)
        else:
            non_sig_pos.append(pos)
            non_sig_scores.append(escape_score)
    assert len(filtered_postions)+len(non_sig_pos) == len(df1)

    print(f"Total Position: {len(filtered_postions) + len(non_sig_pos)} Escape: {len(filtered_postions)} Non escape: {len(non_sig_pos)}")
    #x, y c# color encoding, z is color varieties #s = 1 small size | Make partially transparent (alpha = 0.2)
    width = 8 #Size in inch
    height = 6
    plt.figure(figsize=(width, height))
    non_escape_label = "Non Escape :"+str(len(non_sig_pos))
    plt.hexbin(non_sig_pos, non_sig_scores, gridsize=50, cmap='viridis')
    lable_for_escape_spike = "Escape: "+str(len(filtered_postions))
    plt.scatter(filtered_postions, filtered_scores, c = 'red', s = 0.8, label=lable_for_escape_spike) # alpha=0.2 #c= z, cmap='viridis', 
    # plt.scatter(non_sig_pos, non_sig_scores, c = 'gray', s = 0.1) # alpha=0.2 #c= z, cmap='viridis', 

    plt.xlabel("Escape Positions")
    plt.ylabel("Escape Score")
    legend_font = {'family': 'sans-serif',
               'weight': 'bold',
               'size': 8}
    plt.legend(prop=legend_font)
    # plt.colorbar(label='Z values')
    plt.savefig(hex_plot_save_path)
    print(f"Hexbin plot with scatter plot save to {hex_plot_save_path}")

    plt.show()

    #Save probable escape spikes to csv
    filtered_dataframe.to_csv(escape_spike_save_path, index=False)
    print(f"Escape spikes save to path {escape_spike_save_path}  for escape threshold {escape_threshold}")

In [174]:
#generate_insilico_muated_spikes(insicio_mutated_file_path)
#generate_escape_scores(insicio_mutated_file_path, insicio_mutated_file_escape_path)
#generate_hexbin_plot(hex_plot_save_path, escape_spike_save_path, escape_threshold=0.95)



In [175]:
#get scores for available cases based on filtering 
def compute_save_known_mutants_scores(known_mutatant_path, save_mutant_path):
    #Read csv for which you would like to filter scores
    df_known_mutant = pd.read_csv(known_mutatant_path)
    known_muts  = df_known_mutant['mutant'].to_list()
    
    #read csv that contains all information 
    df_complete = pd.read_csv(insicio_mutated_file_escape_path)
    save_df = pd.DataFrame()
    for mutant in known_muts:
        #get records by mutants and fliter by highest score 
        mutation_mask = df_complete['mutation'] == mutant
        filtered_records_df = df_complete[mutation_mask]
        
        #if records exists in the Db for the mutatant
        if(len(filtered_records_df) > 0):
            #Get the index of row having maximum score and add to the new dataframe 
            max_score_index = filtered_records_df['escape_scores'].idxmax()
            max_escape_score_row = df_complete.iloc[max_score_index]
            # print(max_escape_score_row)
            save_df = save_df.append(max_escape_score_row, ignore_index=True)
        else: 
            print(f"No scores information avaialbe for the mutant {mutant} in csv file {insicio_mutated_file_escape_path} ")
            
    #known_escape_muatants_save_path_with_scores
    save_df.to_csv(save_mutant_path, index=False)
    print("Scores saved to path: ", save_mutant_path)

In [177]:
compute_save_known_mutants_scores(known_mutants_case1_path, known_mutants_case1_path_with_score)
# compute_save_known_mutants_scores(known_mutants_case2_path, known_mutants_case2_path_with_scores)
      
      
        
    
    

No scores information avaialbe for the mutant E484K  in csv file /home/perm/cov/data/additional_escape_variants/gen/insicio_mutated_spikes_with_scores.csv 
No scores information avaialbe for the mutant K417N  in csv file /home/perm/cov/data/additional_escape_variants/gen/insicio_mutated_spikes_with_scores.csv 
No scores information avaialbe for the mutant L452R  in csv file /home/perm/cov/data/additional_escape_variants/gen/insicio_mutated_spikes_with_scores.csv 
No scores information avaialbe for the mutant E484Q  in csv file /home/perm/cov/data/additional_escape_variants/gen/insicio_mutated_spikes_with_scores.csv 


FileNotFoundError: [Errno 2] No such file or directory: 'home/perm/cov/data/mut_cases/case1_scores.csv'

In [180]:
known_mutants_case1_path
!ls home/perm/cov/data/mut_cases/case1_scores.csv

ls: cannot access 'home/perm/cov/data/mut_cases/case1_scores.csv': No such file or directory


38